# Using GPT-4V


This notebook demonstrates how to use GPT-4V's image capabilities directly through the OpenAI API.
We provide helper functions to simplify the creation of prompts and understanding which parameters are available while maintaining the complete flexibility that the API offers.


## Creating Prompts

Prompts for vision enabled models follow the familiar [chat completion](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) format as the non-vision enabled models or requests.

However, including images in the prompt requires a slightly different format. Images are available to the models in two ways: by passing a URL to an image or by passing the base64 encoded image directly in the request.
Note that images can be passed in the `user`, `system` and `assistant` messages, however currently they cannot be in the _first_ message [[source]](https://platform.openai.com/docs/guides/vision).

We can have messages containing text as before, but when we want to include images with a message, `content` becomes a list. That list can contain both text and image messages, in any order. We used the `encode_image` function to convert the image to base64 encoding. The optional `detail` parameter in the `image_url` message specifies the quality of the image. It can be either `low` or `high`. For more details on how images are processed and associated costs, refer to the [OpenAI API documentation](https://platform.openai.com/docs/guides/vision/low-or-high-fidelity-image-understanding). Other providers may not have this functionality.


In [1]:
from pathlib import Path

from not_again_ai.llm.chat_completion.types import (
    ImageContent,
    ImageDetail,
    ImageUrl,
    SystemMessage,
    TextContent,
    UserMessage,
)
from not_again_ai.llm.prompting.compile_prompt import compile_messages, encode_image

sk_infographic = Path.cwd().parent.parent / "tests" / "llm" / "sample_images" / "SKInfographic.png"
sk_diagram = Path.cwd().parent.parent / "tests" / "llm" / "sample_images" / "SKDiagram.png"

messages = [
    SystemMessage(content="You are a helpful {{ persona }}."),
    UserMessage(
        content=[
            TextContent(
                text="Based on these infographics, can you summarize how {{ library }} works in exactly one sentence?"
            ),
            ImageContent(
                image_url=ImageUrl(url=f"data:image/png;base64,{encode_image(sk_infographic)}", detail=ImageDetail.HIGH)
            ),
            ImageContent(
                image_url=ImageUrl(url=f"data:image/png;base64,{encode_image(sk_diagram)}", detail=ImageDetail.LOW)
            ),
        ],
    ),
]

prompt = compile_messages(messages, variables={"persona": "assistant", "library": "Semantic Kernel"})

# Truncate the url fields to avoid cluttering the output
prompt[1].content[1].image_url.url = prompt[1].content[1].image_url.url[0:50] + "..."
prompt[1].content[2].image_url.url = prompt[1].content[2].image_url.url[0:50] + "..."
prompt

[SystemMessage(content='You are a helpful assistant.', role=<Role.SYSTEM: 'system'>, name=None),
 UserMessage(content=[TextContent(type=<ContentPartType.TEXT: 'text'>, text='Based on these infographics, can you summarize how Semantic Kernel works in exactly one sentence?'), ImageContent(type=<ContentPartType.IMAGE: 'image_url'>, image_url=ImageUrl(url='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAADKsA...', detail=<ImageDetail.HIGH: 'high'>)), ImageContent(type=<ContentPartType.IMAGE: 'image_url'>, image_url=ImageUrl(url='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAADWAA...', detail=<ImageDetail.LOW: 'low'>))], role=<Role.USER: 'user'>, name=None)]

Here are the two images that were encoded:

![SKInfographic](https://github.com/DaveCoDev/not-again-ai/blob/main/tests/llm/sample_images/SKInfographic.png?raw=true)

![SKDiagram](https://github.com/DaveCoDev/not-again-ai/blob/main/tests/llm/sample_images/SKDiagram.png?raw=true)


## Making an API Request

With prompt formatted, making the request is easy.

### Simplifying the response format

The response from the API is quite verbose. We can simplify it by extracting only what is needed, depending on the parameters we provided in our request.

Using our helper functions, let's send a request which tries to use all the available parameters. Notice that we use `n=2` to get two completions in one request. However, due to the seed they should always be equivalent. NOTE: We have noticed that the `seed` parameter is hit or miss and does not generate the same completions in all scenarios.


In [2]:
from not_again_ai.llm.chat_completion import chat_completion
from not_again_ai.llm.chat_completion.providers.openai_api import openai_client
from not_again_ai.llm.chat_completion.types import ChatCompletionRequest

client = openai_client()

prompt = compile_messages(messages, variables={"persona": "assistant", "library": "Semantic Kernel"})

request = ChatCompletionRequest(
    messages=prompt,
    model="gpt-4o-mini-2024-07-18",
    max_completion_tokens=200,
    temperature=0.5,
    seed=42,
    n=2,
)
response = chat_completion(request, "openai", client)
response.choices[0].message.content

'Semantic Kernel is a framework that integrates various AI services and plugins to manage and execute tasks by processing prompts, utilizing memory, planning, and invoking functions to deliver results efficiently.'